In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
from helper import *


In [2]:
df = pd.read_csv('data\\simple_ner.csv', encoding = "ISO-8859-1")
#df = df[:100000]
df.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


In [3]:
df = df.fillna(method='ffill')
df['Sentence #'].nunique(), df.Word.nunique(), df.Tag.nunique()

(47959, 35178, 17)

In [4]:
df.groupby('Tag').size().reset_index(name='counts')

,Tag,counts
0,B-art,402
1,B-eve,308
2,B-geo,37644
3,B-gpe,15870
4,B-nat,201
5,B-org,20143
6,B-per,16990
7,B-tim,20333
8,I-art,297
9,I-eve,253


Continue here

In [5]:
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from collections import Counter
import helper

In [6]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s['Word'].values.tolist(), 
                                                           s['POS'].values.tolist(), 
                                                           s['Tag'].values.tolist())]
        self.grouped = self.data.groupby('Sentence #').apply(agg_func)
        self.sentences = [s for s in self.grouped]
        
    def get_next(self):
        try: 
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent += 1
            return s 
        except:
            return None
getter = SentenceGetter(df)
sentences = getter.sentences

In [7]:
X = [sent2features_casual(s) for s in sentences]
y = [sent2labels(s) for s in sentences]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

In [8]:
X_train[0][2]

{'word.lower()': 'came',
 'word.isdigit()': False,
 'postag': 'VBD',
 'postag[:2]': 'VB',
 '-1:word.lower()': 'it',
 '-1:postag': 'PRP',
 '-1:postag[:2]': 'PR',
 '-2:postag': 'RB',
 '-2:postag[:2]': 'RB',
 '+1:word.lower()': 'to',
 '+1:postag': 'TO',
 '+1:postag[:2]': 'TO',
 '+2:postag': 'VB',
 '+2:postag[:2]': 'VB'}

In [9]:
crf = sklearn_crfsuite.CRF(
    algorithm='l2sgd',
    #c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

C:\Users\ACER\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:209: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


CRF(algorithm='l2sgd', all_possible_transitions=True, c2=0.1,
    keep_tempfiles=None, max_iterations=100)

In [18]:
all_tags = df.Tag.values

In [19]:
classes = np.unique(all_tags)
classes = classes.tolist()

In [20]:
new_classes = classes.copy()
new_classes

['B-art',
 'B-eve',
 'B-geo',
 'B-gpe',
 'B-nat',
 'B-org',
 'B-per',
 'B-tim',
 'I-art',
 'I-eve',
 'I-geo',
 'I-gpe',
 'I-nat',
 'I-org',
 'I-per',
 'I-tim',
 'O']

In [21]:
y_pred = crf.predict(X_test)
print(metrics.flat_classification_report(y_test, y_pred, labels = new_classes))

C:\Users\ACER\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass labels=['B-art', 'B-eve', 'B-geo', 'B-gpe', 'B-nat', 'B-org', 'B-per', 'B-tim', 'I-art', 'I-eve', 'I-geo', 'I-gpe', 'I-nat', 'I-org', 'I-per', 'I-tim', 'O'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


              precision    recall  f1-score   support

       B-art       0.44      0.11      0.18       143
       B-eve       0.49      0.34      0.40       106
       B-geo       0.84      0.92      0.88     12447
       B-gpe       0.97      0.92      0.95      5284
       B-nat       0.83      0.31      0.45        78
       B-org       0.82      0.70      0.75      6615
       B-per       0.86      0.82      0.84      5652
       B-tim       0.92      0.88      0.90      6856
       I-art       0.12      0.04      0.06       105
       I-eve       0.29      0.22      0.25        93
       I-geo       0.80      0.82      0.81      2520
       I-gpe       0.88      0.55      0.68        69
       I-nat       1.00      0.35      0.52        23
       I-org       0.83      0.77      0.80      5597
       I-per       0.85      0.91      0.88      5674
       I-tim       0.84      0.72      0.78      2207
           O       0.99      0.99      0.99    291891

    accuracy              

In [22]:
new_classes.remove('O')

In [23]:
y_pred = crf.predict(X_test)
print(metrics.flat_classification_report(y_test, y_pred, labels = new_classes))

C:\Users\ACER\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass labels=['B-art', 'B-eve', 'B-geo', 'B-gpe', 'B-nat', 'B-org', 'B-per', 'B-tim', 'I-art', 'I-eve', 'I-geo', 'I-gpe', 'I-nat', 'I-org', 'I-per', 'I-tim'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


              precision    recall  f1-score   support

       B-art       0.44      0.11      0.18       143
       B-eve       0.49      0.34      0.40       106
       B-geo       0.84      0.92      0.88     12447
       B-gpe       0.97      0.92      0.95      5284
       B-nat       0.83      0.31      0.45        78
       B-org       0.82      0.70      0.75      6615
       B-per       0.86      0.82      0.84      5652
       B-tim       0.92      0.88      0.90      6856
       I-art       0.12      0.04      0.06       105
       I-eve       0.29      0.22      0.25        93
       I-geo       0.80      0.82      0.81      2520
       I-gpe       0.88      0.55      0.68        69
       I-nat       1.00      0.35      0.52        23
       I-org       0.83      0.77      0.80      5597
       I-per       0.85      0.91      0.88      5674
       I-tim       0.84      0.72      0.78      2207

   micro avg       0.86      0.84      0.85     53469
   macro avg       0.74   

In [24]:
from collections import Counter
def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))
print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))
print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])

Top likely transitions:
I-org  -> I-org   8.570730
B-geo  -> I-geo   8.474305
B-org  -> I-org   8.421875
B-tim  -> I-tim   7.769305
B-per  -> I-per   7.729057
B-art  -> I-art   7.511492
I-tim  -> I-tim   7.388941
I-per  -> I-per   7.200962
I-art  -> I-art   7.198596
O      -> O       6.894479
I-eve  -> I-eve   6.685049
B-eve  -> I-eve   6.589331
B-gpe  -> I-gpe   6.514260
I-geo  -> I-geo   6.442124
B-nat  -> I-nat   5.558473
I-gpe  -> I-gpe   5.235349
O      -> B-per   5.100205
O      -> B-geo   4.718776
O      -> B-tim   4.637727
O      -> B-org   4.179881

Top unlikely transitions:
I-eve  -> B-tim   -2.047972
B-gpe  -> I-geo   -2.078873
I-per  -> B-geo   -2.083811
B-org  -> I-per   -2.091779
B-gpe  -> I-org   -2.176951
I-eve  -> B-eve   -2.293871
O      -> I-art   -2.441563
O      -> I-per   -2.638834
I-geo  -> B-geo   -2.799409
B-org  -> B-org   -2.993215
O      -> I-tim   -3.174125
O      -> I-org   -3.184291
B-geo  -> B-geo   -3.284742
I-org  -> B-org   -3.352645
I-tim  -> B-tim  

In [25]:
import eli5
eli5.show_weights(crf, top=10)

From \ To,O,B-art,I-art,B-eve,I-eve,B-geo,I-geo,B-gpe,I-gpe,B-nat,I-nat,B-org,I-org,B-per,I-per,B-tim,I-tim
O,6.894,2.422,-2.442,3.539,-2.035,4.719,-3.567,3.419,-1.228,2.561,-1.207,4.18,-3.184,5.1,-2.639,4.638,-3.174
B-art,-0.187,-0.202,7.511,-0.131,-0.277,0.328,-0.287,-1.299,-0.319,-0.118,-0.161,0.705,-0.486,-1.566,-0.529,-0.094,-0.536
I-art,-0.647,-0.381,7.199,-0.189,-0.19,0.425,-0.372,-0.952,-0.343,-0.053,-0.075,-0.881,-0.275,-0.112,-0.442,-1.516,-0.21
B-eve,-0.505,-0.131,-0.211,-0.433,6.589,-1.177,-0.328,-0.949,-0.246,-0.369,-0.169,-1.279,-0.439,-1.429,-0.393,0.415,-0.411
I-eve,0.284,-0.072,-0.08,-2.294,6.685,-0.562,-0.189,-0.392,-0.162,-0.029,-0.052,-0.52,-0.122,-0.707,-0.263,-2.048,-0.419
B-geo,1.846,1.154,-0.864,-0.772,-1.181,-3.285,8.474,1.343,-1.899,-0.455,-0.689,1.062,-1.951,-0.014,-1.811,2.142,-1.455
I-geo,0.246,1.846,-0.793,-0.731,-0.936,-2.799,6.442,-1.915,-1.537,-0.214,-0.358,0.027,-1.329,-0.281,-1.413,0.599,-1.476
B-gpe,3.119,-1.164,-0.865,-1.017,-1.795,0.599,-2.079,-4.131,6.514,-0.837,-0.673,2.207,-2.177,1.419,-1.845,0.471,-1.056
I-gpe,-0.135,-0.444,-0.079,-0.076,-0.201,-0.429,-0.494,-1.002,5.235,-0.064,-0.041,-1.401,-0.279,-0.355,-0.393,-1.313,-0.309
B-nat,-0.526,-0.052,-0.077,-0.069,-0.161,-0.438,-0.092,-0.697,-0.205,-0.772,5.558,-0.463,-0.132,-0.349,-0.302,-1.256,-0.27


In [19]:
import pickle
file_name = 'casual_test.pkl'
pickle.dump(crf,open(file_name,'wb'))